## Descripción Pedagógica de la Actividad del Notebook

**Título:** Actividad Práctica: Del API a la Base de Datos

**Resumen:**
Esta actividad guiará a los estudiantes en un proceso paso a paso para consumir datos de una API web (la API de Dragon Ball) y almacenarlos de forma persistente en una base de datos local (SQLite). El objetivo es que los estudiantes comprendan el flujo completo de datos, desde su obtención en la web hasta su almacenamiento y consulta en un entorno controlado.

**Objetivos de Aprendizaje:**
Al finalizar esta actividad, los estudiantes serán capaces de:
* Conectarse a una API web y obtener datos en formato JSON.
* Diseñar un esquema de base de datos simple basado en datos del mundo real.
* Crear y gestionar una base de datos SQLite utilizando Python.
* Realizar operaciones CRUD (Crear, Leer, Actualizar, Eliminar) básicas en una base de datos.
* Comprender la importancia de las consultas parametrizadas para la seguridad (prevención de inyección SQL).
* Aplicar conceptos de programación y bases de datos en un proyecto práctico y motivador.

**Enfoque Pedagógico:**
El enfoque se basa en el 'aprendizaje basado en proyectos' y el 'aprendizaje activo'. En lugar de teoría abstracta, los estudiantes trabajan con datos reales y relevantes (personajes de Dragon Ball) para mantener la motivación. La estructura del notebook está diseñada para ser secuencial y progresiva, permitiendo a los estudiantes construir su conocimiento de forma incremental. Cada sección combina una breve explicación teórica con un bloque de código práctico, fomentando la experimentación y el aprendizaje autónomo.

**Relevancia y Aplicación:**
Las habilidades adquiridas en esta actividad son fundamentales en el desarrollo de software moderno. La capacidad de interactuar con APIs y gestionar bases de datos es esencial para crear aplicaciones web, móviles y de escritorio. Este ejercicio sirve como una base sólida para el proyecto final del curso, donde los estudiantes podrán aplicar estas técnicas para crear una aplicación más compleja que integre múltiples tecnologías (Flask, pandas, matplotlib, etc.).

In [ ]:
%pip install requests

In [1]:
import sqlite3
import requests
import json

## 1. Introducción a SQLite y Conceptos de Bases de Datos (20 minutos)

SQLite es un sistema de gestión de bases de datos relacionales (RDBMS) ligero, sin servidor y autocontenido. Esto significa que no necesita un proceso de servidor separado para funcionar, y toda la base de datos se almacena en un único archivo en disco.

**¿Por qué usar SQLite?**
- **Ligero:** Ocupa muy poco espacio en disco y memoria.
- **Sin servidor:** No requiere configuración de un servidor de base de datos.
- **Autocontenido:** La base de datos completa reside en un solo archivo.
- **Fácil de usar:** Ideal para aplicaciones locales, prototipos y pruebas.

**Terminología Básica de Bases de Datos:**
- **Tabla:** Una colección de datos relacionados organizados en filas y columnas.
- **Fila (Registro/Tupla):** Una única entrada de datos en una tabla.
- **Columna (Campo/Atributo):** Una categoría específica de datos dentro de una tabla.
- **Clave Primaria (Primary Key):** Una columna (o conjunto de columnas) que identifica de forma única cada fila en una tabla. No puede contener valores nulos y debe ser única.

## 2. Configuración del Entorno (10 minutos)

Necesitamos importar las librerías `sqlite3` (que viene con Python) y `requests` (para las llamadas a la API). Si no tienes `requests` instalado, puedes hacerlo con `pip install requests` en tu terminal.

## 3. Conectando a una Base de Datos SQLite (20 minutos)

Para interactuar con una base de datos SQLite, primero necesitamos establecer una conexión. Si el archivo de la base de datos no existe, `sqlite3.connect()` lo creará automáticamente.

In [2]:
DB_NAME = 'dragonball_characters.db'

try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Conexión a la base de datos '{DB_NAME}' establecida con éxito.")
except sqlite3.Error as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión a la base de datos 'dragonball_characters.db' establecida con éxito.


## 4. Obteniendo Datos de la API de Dragon Ball (30 minutos)

Vamos a utilizar la API de Dragon Ball para obtener una lista de personajes. Recordaremos cómo hacer una solicitud GET y cómo procesar la respuesta JSON.

In [3]:
API_URL = "https://dragonball-api.com/api/characters?page=2&limit=5"

try:
    response = requests.get(API_URL)
    response.raise_for_status()
    data = response.json()
    characters = data.get('items', [])

    print(f"Se obtuvieron {len(characters)} personajes de la API.")
    if characters:
        print("Primer personaje obtenido:")
        print(json.dumps(characters[0], indent=2))
    else:
        print("No se encontraron personajes.")

except requests.exceptions.RequestException as e:
    print(f"Error al conectar con la API: {e}")
except json.JSONDecodeError:
    print("Error al decodificar la respuesta JSON de la API.")

Se obtuvieron 5 personajes de la API.
Primer personaje obtenido:
{
  "id": 6,
  "name": "Zarbon",
  "ki": "20.000",
  "maxKi": "30.000",
  "race": "Frieza Race",
  "gender": "Male",
  "description": "Zarbon es uno de los secuaces de Freezer y un luchador poderoso.",
  "image": "https://dragonball-api.com/characters/zarbon.webp",
  "affiliation": "Army of Frieza",
  "deletedAt": null
}


## 5. Diseñando el Esquema de la Base de Datos (20 minutos)

Antes de crear la tabla, debemos decidir qué información de los personajes queremos almacenar y qué tipo de datos tendrá cada columna en SQLite.

Considerando la estructura de los datos de la API, podemos definir las siguientes columnas para nuestra tabla `characters`:

- `id`: INTEGER PRIMARY KEY (Identificador único del personaje)
- `name`: TEXT (Nombre del personaje)
- `race`: TEXT (Raza del personaje)
- `description`: TEXT (Descripción del personaje)
- `image`: TEXT (URL de la imagen del personaje)
- `ki`: TEXT (Nivel de Ki del personaje)
- `maxKi`: TEXT (Ki máximo del personaje)
- `affiliation`: TEXT (Afiliación del personaje)

**Tipos de Datos SQLite:**
- `NULL`: El valor es un valor NULL.
- `INTEGER`: El valor es un entero con signo, almacenado en 1, 2, 3, 4, 6 u 8 bytes dependiendo de la magnitud del valor.
- `REAL`: El valor es un valor de punto flotante, almacenado como un número de punto flotante IEEE 754 (8 bytes).
- `TEXT`: El valor es una cadena de texto, almacenada usando la codificación de la base de datos (UTF-8, UTF-16BE o UTF-16LE).
- `BLOB`: El valor es un blob de datos, almacenado exactamente como se introdujo.

## 6. Creando una Tabla (20 minutos)

Ahora, usaremos un comando SQL `CREATE TABLE` para definir la estructura de nuestra tabla `characters` en la base de datos.

In [4]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS characters (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    race TEXT,
    description TEXT,
    image TEXT,
    ki TEXT,
    maxKi TEXT,
    affiliation TEXT
);
"""

try:
    cursor.execute(create_table_sql)
    conn.commit()
    print("Tabla 'characters' creada o ya existente.")
except sqlite3.Error as e:
    print(f"Error al crear la tabla: {e}")

Tabla 'characters' creada o ya existente.


## 7. Insertando Datos en la Tabla (30 minutos)

Ahora que tenemos la tabla, vamos a insertar los datos de los personajes que obtuvimos de la API. Es crucial usar consultas parametrizadas (`?`) para evitar problemas de seguridad (inyección SQL) y manejar correctamente los tipos de datos.

In [5]:
insert_sql = """
INSERT INTO characters (id, name, race, description, image, ki, maxKi, affiliation)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
ON CONFLICT(id) DO UPDATE SET
    name = excluded.name,
    race = excluded.race,
    description = excluded.description,
    image = excluded.image,
    ki = excluded.ki,
    maxKi = excluded.maxKi,
    affiliation = excluded.affiliation;
"""

if characters:
    for char in characters:
        try:
            char_data = (
                char.get('id'),
                char.get('name'),
                char.get('race'),
                char.get('description'),
                char.get('image'),
                char.get('ki'),
                char.get('maxKi'),
                char.get('affiliation')
            )
            cursor.execute(insert_sql, char_data)
        except sqlite3.Error as e:
            print(f"Error al insertar personaje {char.get('name')}: {e}")
    conn.commit()
    print(f"Se insertaron/actualizaron {len(characters)} personajes en la base de datos.")
else:
    print("No hay personajes para insertar.")

Se insertaron/actualizaron 5 personajes en la base de datos.


## 8. Consultando Datos de la Base de Datos (20 minutos)

Una vez que los datos están en la base de datos, podemos recuperarlos usando sentencias `SELECT` de SQL. Aprenderemos a seleccionar todas las columnas, columnas específicas y a filtrar resultados.

In [6]:
print("\n--- Todos los personajes ---")
cursor.execute("SELECT id, name, race FROM characters")
all_characters = cursor.fetchall()
for char in all_characters:
    print(char)

print("\n--- Personajes Saiyan ---")
cursor.execute("SELECT name, ki FROM characters WHERE race = ?", ('Saiyan',))
saiyans = cursor.fetchall()
for saiyan in saiyans:
    print(saiyan)

print("\n--- Personaje con ID 1 ---")
cursor.execute("SELECT name, description FROM characters WHERE id = ?", (1,))
char_id_1 = cursor.fetchone()
if char_id_1:
    print(char_id_1)
else:
    print("Personaje no encontrado.")


--- Todos los personajes ---
(6, 'Zarbon', 'Frieza Race')
(7, 'Dodoria', 'Frieza Race')
(8, 'Ginyu', 'Frieza Race')
(9, 'Celula', 'Android')
(10, 'Gohan', 'Saiyan')

--- Personajes Saiyan ---
('Gohan', '45.000.000')

--- Personaje con ID 1 ---
Personaje no encontrado.


## 9. Actualizando y Eliminando Datos (Opcional/Extensión - 10 minutos)

Para una gestión completa de la base de datos, es útil saber cómo modificar (`UPDATE`) y eliminar (`DELETE`) registros existentes.

In [7]:
print("\n--- Actualizando personaje ---")
update_sql = "UPDATE characters SET description = ? WHERE name = ?"
try:
    cursor.execute(update_sql, ("Un guerrero legendario con un gran corazón.", "Goku"))
    conn.commit()
    print("Descripción de Goku actualizada.")
except sqlite3.Error as e:
    print(f"Error al actualizar: {e}")

print("\n--- Eliminando personaje ---")
delete_sql = "DELETE FROM characters WHERE name = ?"
try:
    cursor.execute(delete_sql, ("Freeza",))
    conn.commit()
    print("Personaje Freeza eliminado (si existía).")
except sqlite3.Error as e:
    print(f"Error al eliminar: {e}")

print("\n--- Verificando cambios ---")
cursor.execute("SELECT name, description FROM characters WHERE name = 'Goku'")
print("Goku después de actualizar:", cursor.fetchone())
cursor.execute("SELECT name FROM characters WHERE name = 'Freeza'")
print("Freeza después de eliminar:", cursor.fetchone())


--- Actualizando personaje ---
Descripción de Goku actualizada.

--- Eliminando personaje ---
Personaje Freeza eliminado (si existía).

--- Verificando cambios ---
Goku después de actualizar: None
Freeza después de eliminar: None


## 10. Cerrando la Conexión a la Base de Datos (5 minutos)

Es una buena práctica cerrar la conexión a la base de datos cuando ya no se necesita para liberar recursos.

In [8]:
if conn:
    conn.close()
    print("Conexión a la base de datos cerrada.")

Conexión a la base de datos cerrada.


## 11. Resumen y Próximos Pasos (5 minutos)

En este laboratorio, hemos aprendido los fundamentos de SQLite con Python, desde la conexión y creación de tablas hasta la inserción y consulta de datos, utilizando información real de una API.

**Posibles Mejoras y Extensiones:**
- **Consultas más complejas:** Explorar `JOIN`s, funciones de agregación (`COUNT`, `SUM`, `AVG`).
- **Visualización de datos:** Usar `pandas` y `matplotlib` para analizar y graficar los datos almacenados.
- **Aplicación Web:** Construir una pequeña aplicación web con `Flask` que muestre los personajes de la base de datos.
- **Manejo de errores avanzado:** Implementar un manejo de errores más robusto.
- **Más datos de la API:** Obtener más páginas de personajes o usar el endpoint `GetByCharacter` para detalles específicos.